In [ ]:
import os, re
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
from datetime import datetime
from uuid import uuid4

# --- Directory setup ---
INPUT_DIR = "/content/drive/MyDrive/sec_filings_raw"
OUTPUT_DIR_CLEAN = "data/clean"
os.makedirs(OUTPUT_DIR_CLEAN, exist_ok=True)

# ------------------------------------------------------------
# Helper: Clean and normalize text
# ------------------------------------------------------------
def clean_and_normalize_text(content):
    try:
        text = BeautifulSoup(content, "html.parser").get_text(" ")
    except Exception:
        text = content
    text = re.sub(r'[\t\n\r\f]+', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text.upper()

# ------------------------------------------------------------
# Helper: Detect filing type
# ------------------------------------------------------------
def detect_form_type(file_path):
    lower_path = file_path.lower()
    if "10-k" in lower_path:
        return "10K"
    elif "10q" in lower_path or "10-q" in lower_path:
        return "10Q"
    else:
        return "OTHER"

# ------------------------------------------------------------
# Helper: Extract filing date from content or path
# ------------------------------------------------------------
import re
from datetime import datetime

def is_valid_yyyymmdd(date_str):
    """Check whether an 8-digit string like 20250630 is a valid date between 1993 and 2025."""
    if not re.fullmatch(r"\d{8}", date_str):
        return False

    # --- Rule 1: starts with 19 or 20 ---
    if not (date_str.startswith("19") or date_str.startswith("20")):
        return False

    year = int(date_str[:4])
    month = int(date_str[4:6])
    day = int(date_str[6:8])

    # --- Rule 2: year range ---
    if not (1993 <= year <= 2025):
        return False

    # --- Rule 3: month range ---
    if not (1 <= month <= 12):
        return False

    # --- Rule 4: day range (and actual calendar validity) ---
    try:
        datetime(year, month, day)  # will raise ValueError if invalid day/month combo
    except ValueError:
        return False

    return True


def extract_filing_date(file_content, file_path=None):
    """
    Extract the most likely filing date from SEC text content.
    Finds all 8-digit numbers, validates them, and returns the first real one.
    """

    # --- Find all 8-digit sequences ---
    candidates = re.findall(r"\b\d{8}\b", file_content)
    valid_dates = []

    for c in candidates:
        if is_valid_yyyymmdd(c):
            valid_dates.append(c)

    if valid_dates:
        # pick earliest or latest as you prefer
        chosen = valid_dates[0]
        return datetime.strptime(chosen, "%Y%m%d").strftime("%Y-%m-%d")

    # fallback: maybe filename has date pattern
    if file_path:
        match = re.search(r"(\d{4}-\d{2}-\d{2})", file_path)
        if match:
            return match.group(1)

    return None


# ------------------------------------------------------------
# Collect all text/html files
# ------------------------------------------------------------
all_files_for_cleaning = []
for root, _, files in os.walk(INPUT_DIR):
    for filename in files:
        if filename.lower().endswith((".txt", ".html", ".htm")):
            all_files_for_cleaning.append(os.path.join(root, filename))

print(f"🧾 Found {len(all_files_for_cleaning)} files to clean.")

# ------------------------------------------------------------
# Process and clean each file
# ------------------------------------------------------------
records = []
processed_count = 0

for file_path in tqdm(all_files_for_cleaning, desc="Cleaning Text"):
    try:
        with open(file_path, 'r', encoding='latin-1', errors='ignore') as f:
            raw_content = f.read()

        if not raw_content.strip():
            print(f"⚠️ Empty file: {file_path}")
            continue

        # --- Clean text ---
        clean_text = clean_and_normalize_text(raw_content)

        # --- Extract metadata ---
        path_parts = file_path.split(os.sep)
        cik_str = path_parts[-4]  # Adjust if your folder depth differs
        form_type = detect_form_type(file_path)
        filing_date = extract_filing_date(raw_content, file_path)

        # --- Fallback name if no date found ---
        if filing_date:
            new_filename = f"{filing_date}.txt"
        else:
            new_filename = f"unknown_{uuid4().hex[:8]}.txt"
            filing_date = "unknown"

        # --- Save cleaned text file ---
        clean_cik_dir = os.path.join(OUTPUT_DIR_CLEAN, cik_str, form_type)
        os.makedirs(clean_cik_dir, exist_ok=True)

        clean_file_path = os.path.join(clean_cik_dir, new_filename)
        with open(clean_file_path, 'w', encoding='utf-8') as f:
            f.write(clean_text)

        # --- Add record for Excel ---
        records.append({
            "Ticker/CIK": cik_str,
            "Form_Type": form_type,
            "Filing_Date": filing_date,
            "File_Name": new_filename
        })

        processed_count += 1

    except Exception as e:
        print(f"❌ Error processing {file_path}: {e}")

# ------------------------------------------------------------
# Save metadata to Excel
# ------------------------------------------------------------
df = pd.DataFrame(records)
excel_path = os.path.join(OUTPUT_DIR_CLEAN, "filing_metadata.xlsx")
df.to_excel(excel_path, index=False)

print(f"\n✅ Total {processed_count} files cleaned and organized into {OUTPUT_DIR_CLEAN}.")
print(f"💾 Metadata saved at: {excel_path}")


In [2]:
# fetching the stock price 

In [1]:
# ================================================
# 1. Load Dataset
# ================================================
import pandas as pd
import numpy as np
import yfinance as yf

df = pd.read_csv("final_all_column (1).csv")

print("Dataset loaded:", df.shape)

# ================================================
# 2. Convert FilingDate to Datetime
# ================================================
df["FILING_DATE"] = pd.to_datetime(df["FILING_DATE"], errors="coerce")

print("Parsed filing dates. Null count:", df["FILING_DATE"].isna().sum())

# ================================================
# 3. Clean Ticker Column
# ================================================
df["Ticker"] = df["Ticker"].astype(str).str.strip()
tickers = df["Ticker"].dropna().unique().tolist()

print("Unique tickers found:", len(tickers))

# ================================================
# 4. Download ALL Stock Prices at Once (FAST)
# ================================================
print("Downloading daily stock prices... this may take 1–2 minutes.")

price_data = yf.download(
    tickers,
    start="1980-01-01",
    end="2025-12-31",
    interval="1d",
    auto_adjust=True,
    group_by='ticker',
    progress=True
)

print("Stock prices downloaded successfully.")

# ================================================
# 5. Helper Functions
# ================================================
def get_price_on_or_after(ticker, target_date):
    """Return the first available price on or after target_date."""
    try:
        prices = price_data[ticker]["Close"]
        valid_dates = prices.index[prices.index >= target_date]
        if len(valid_dates) == 0:
            return np.nan
        return prices.loc[valid_dates[0]]
    except:
        return np.nan


def get_return_30d(ticker, date):
    """Return 30d return based on adjusted close prices."""
    try:
        date_30 = date + pd.Timedelta(days=30)
        p0 = get_price_on_or_after(ticker, date)
        p30 = get_price_on_or_after(ticker, date_30)
        if pd.isna(p0) or pd.isna(p30):
            return np.nan
        return (p30 - p0) / p0
    except:
        return np.nan

# ================================================
# 6. Compute Return_30D for Each Filing
# ================================================
print("Computing 30-day returns...")

df["Return_30D_new"] = df.apply(
    lambda row: get_return_30d(row["Ticker"], row["FILING_DATE"]),
    axis=1
)

print("Finished computing new 30-day return values.")

# ================================================
# 7. Save Output
# ================================================
output_path = "final_with_return30d.csv"
df.to_csv(output_path, index=False)

print(f"File saved successfully: {output_path}")

# ================================================
# 8. Show sample
# ================================================
df.head()


ModuleNotFoundError: No module named 'yfinance'